


**implementing the k-means clustering algorithm to cluster passages**

The collections from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset have been used in this project.

**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer. The terms in the passage text can be accessed by splitting the text based on space.

**Centroid vector values**

Each row is a tab separated entry where the first column is cluster id and the second column is the vector which is used to initialize the cluster centroid in the k-means Algorithm. 


In [1]:

import os
import numpy as np
import pandas as pd
#Setting the input file
passage = "passages.tok.clean_kstem"
init_centroid_file = "centroid.txt"


In [2]:
''' 
In this function, load the initial centroid values for k clusters.  
Return Variables: 
init_centroid_vec - array where each element corresponds to the centroid vector. 
'''

# load initial centroid values
def load_centroid_init(init_centroid_file):
  init_centroid_vec = []
  for line in open(init_centroid_file):
   row = line.strip().split('\t')
   init_centroid_vec.append(np.fromstring(row[1][1:-1],sep=',').astype(float))
  return init_centroid_vec 


''' 
In this function, iterate through the input passage and load the data  
Return Variables: 
vocab - dict mapping word to an integer [0,len(vocab)]
df - dict mapping word to document frquency
num_passages - total number of passages in the input collection
docs - dict mapping each passage id to unique integer between [0,len(num_passages)]
'''
# load vocabulary
def load_vocab(passage):
  df = {}
  vocab = {}
  count = 0
  doc_count = 0
  num_passages = 0
  docs = {}
  for line in open(passage):  
   row = line.strip().split('\t')
   docs[row[0]] = doc_count
   doc_count+=1
   terms = row[1].split(' ')
   num_passages+=1
   for word in set(terms):
    if word not in df:
      df[word]=0
    df[word]+=1
   for word in terms:  
    if word not in vocab: 
      vocab[word]=count
      count+=1 

  return vocab,df,num_passages,docs,doc_count     

init_centroid_vec = load_centroid_init(init_centroid_file)
vocab, df, num_passages, docs,doc_count = load_vocab(passage)

print('Vocabulary Size : {0}'.format(len(vocab)))
print('Total Number of Passages: {0}'.format(num_passages))


Vocabulary Size : 3630
Total Number of Passages: 500



Create an input data matrix $P = m \times n$  where $P_{i,j}=tf{\text -}idf(i,j)$. The definition of the $tf{\text -}idf(i,j)$ formulation is given below. 

$$ tf{\text -}idf(i,j) = ln(1+count(i,j)) ln(1 + \frac{|C|}{df(j)}) $$


$tf{\text -}idf(i,j)$: the  $tf{\text -}idf$ value of the word corresponding to integer $j$ and passage corresponding to integer $i$ as defined in vocab and docs data structures, respectively. 

$count(i,j)$ = number of times word corresponding to integer $j$ occurs in passage corresponding to integer $i$.

$df(j)$ - The document frequency of word corresponding to integer $j$ (i.e., the number of documents that contain the $j^{th}$ word.

$|C|$ - Total number of passages in the collection.


In [3]:
''' 
In this function, create input matrix  
Return Variables: 
pass_matrix - matrix of shape m by n where each row corresponds to a passage and each column to a word
'''
import math

def create_input_matrix(passage,df,num_passages,vocab):
  #create a dictionary from passage, which maps passage id to list of its words
  passage_dict = {}
  for line in open(passage):
    row = line.strip().split('\t')
    passage_dict[row[0]]=row[1]
  #create a tf-idf matrix
  pass_matrix= np.zeros((num_passages, len(vocab)), dtype="float")
  C=num_passages
  for id_original, id_integer in docs.items():
    words= passage_dict[id_original].split(' ')
    for word in words:
      pass_matrix[id_integer][vocab[word]]= (math.log(1+words.count(word)))*(math.log(1+(C/df[word])))

  return pass_matrix

pass_matrix =  create_input_matrix(passage,df,num_passages,vocab)   

print('Shape of the input matrix : {0}'.format(np.shape(pass_matrix)))  


Shape of the input matrix : (500, 3630)


In [4]:
pass_matrix

array([[1.18964762, 2.60167109, 2.87728161, ..., 0.        , 0.        ,
        0.        ],
       [0.75058408, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.37929524, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 4.30902299,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        4.30902299]])


implementing the k-means clustering algorithm with k=3(number of clusters). 


In [5]:
# centroid is a matrix instantiated with initial centroid values
''' 
In this function, centroid is instantiated with initial centroid values
Here we assume for k=3, the cluster ids are {0,1,2}   
Return Variables: 
centroid - matrix where rows correspond to clusters and columns correspond to words. 
'''
def  init_centers(vocab, k, init_centroid_vec):
  vocab_size = len(vocab)
  centroid = np.zeros((k, vocab_size), dtype='float')
  for i in range(len(init_centroid_vec)):
     centroid[i] = init_centroid_vec[i]
  return centroid   

''' 
In this function, implement kmeans algorithm   
Return Variables: 
final_cluster_assignment - array with cluster id values indexed by the passageid (integer mappings in docs)
num_passage_cluster_final - dict with cluster-id and key and number of elements in the cluster as value
'''
def kmeans(num_passages, pass_matrix, centroid, k):
  iteration=30
  for iter in range(iteration):
    #create a matrix to calculate the euclidean distance from each passage id and centroid
    distance_matrix=np.zeros((num_passages,k))
    for i in range(len(centroid)):
      for j in range(len(pass_matrix)):
        distance_matrix[j][i]=np.sqrt(np.sum((pass_matrix[j]-centroid[i])**2))
    #assign cluster labels
    cluster_labels = np.argmin(distance_matrix, axis = 1)
    #update centroid at each iteration
    centroid = np.asarray([pass_matrix[cluster_labels == i].mean(axis = 0) for i in range(k)])
  cluster_labels_dict={}
  for each_label in range(len(cluster_labels)):
    if cluster_labels[each_label] not in cluster_labels_dict:
      cluster_labels_dict[cluster_labels[each_label]]=[]
      cluster_labels_dict[cluster_labels[each_label]].append(each_label)
    else:
      cluster_labels_dict[cluster_labels[each_label]].append(each_label)
  
  final_cluster_assignment_dict={}
  for passageid, id_integer in docs.items():
    for clusterid, id_integer1 in cluster_labels_dict.items():
      for each_integer in id_integer1:
        if each_integer == id_integer:
          if clusterid not in final_cluster_assignment_dict:
            final_cluster_assignment_dict[clusterid]=[]
            final_cluster_assignment_dict[clusterid].append(passageid)
          else:
            final_cluster_assignment_dict[clusterid].append(passageid)
  # create an array with cluster id values indexed by the passageid
  final_cluster_assignment=np.array(final_cluster_assignment_dict.values())

  num_passage_cluster_final={}
  for each_cluster_id,passage_ids in final_cluster_assignment_dict.items():
    num_passage_cluster_final[each_cluster_id]=len(passage_ids)
    
  return final_cluster_assignment,num_passage_cluster_final,final_cluster_assignment_dict

centroid = init_centers(vocab, 3, init_centroid_vec)
final_cluster_assignment, num_passage_cluster_final,final_cluster_assignment_dict = kmeans(num_passages, pass_matrix, centroid, 3)

print('Number of initial centroids : {0}'.format(len(centroid)) )
print("----------------------------------")
print("number of elements in each cluster:")

for cid,cnum in num_passage_cluster_final.items():
   print(cid, cnum)  



Number of initial centroids : 3
----------------------------------
number of elements in each cluster:
1 128
0 363
2 9


**Evaluation**

1) IntraCluster Similarity Metric - Average Diameter 
Distance of each cluster $S$

$$Sim(S) = \frac{1}{|S| (|S|-1)} \sum_{x,y \in S,x \neq y} dist(x,y)$$

$|S|$ - number of passages assigned to cluster $S$

$dist(x,y)$ - the squared euclidean distance between passages $x$ and $y$.


2) Intercluster Similarity Metric - 
Average Linkage Distance between a pair of clusters $S,T$

$$Sim(S,T) = \frac{1}{|S| |T|} \sum_{x \in S,y \in T} dist(x,y)$$

$|S|$ - number of passages assigned to cluster $S$

$|T|$ - number of passages assigned to cluster $T$

$dist(x,y)$ - the squared euclidean distance between passages $x$ and $y$.


In [6]:
''' 
In this function, implement intracluster similarity metric Average Diameter Distance   
Return Variables: 
sim_score - dict with metric score corresponding to each of the three clusters
'''
def average_diameter_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix,final_cluster_assignment_dict):
  sim_score={}
  for clusterid , passages in final_cluster_assignment_dict.items():
    sigma=0
    for each_passage_i in passages:
      for each_passage_j in passages:
        #not for the same passageid
        if each_passage_i != each_passage_j:
          #claculate distance of each passageid in each cluster
          sigma+=(np.sum(np.square((pass_matrix[docs[each_passage_i]]
                                    -pass_matrix[docs[each_passage_j]]))))
    result=sigma/(num_passage_cluster_final[clusterid]
                  *(num_passage_cluster_final[clusterid]-1))
    sim_score[clusterid]=result
   
  return sim_score

''' 
In this function, implement intercluster similarity metric Average Linkage Distance   
Return Variables: 
sim_score - dict with metric score corresponding to each pair of clusters
'''
def average_linkage_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix,final_cluster_assignment_dict):
  sim_score={}
  for clusterid , passages in final_cluster_assignment_dict.items():
    for clusterid_second , passages_second in final_cluster_assignment_dict.items():
      sigma=0
      if clusterid != clusterid_second:
        for each_passage_i in passages:
          for each_passage_j in passages_second:
            if each_passage_i != each_passage_j:
              sigma+=(np.sum(np.square((pass_matrix[docs[each_passage_i]]
                                        -pass_matrix[docs[each_passage_j]]))))
      #similarity for pairs of cluster
      if clusterid != clusterid_second and (clusterid_second, clusterid) not in sim_score:
        result = sigma/(num_passage_cluster_final[clusterid]
                        *(num_passage_cluster_final[clusterid_second]))

        sim_score[(clusterid, clusterid_second)]=result
   
  return sim_score

intra_score  = average_diameter_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix,final_cluster_assignment_dict) 
inter_score =  average_linkage_dist(num_passage_cluster_final,final_cluster_assignment,pass_matrix,final_cluster_assignment_dict)  

print("intracluster scores for very cluster:")
for cid,score in intra_score.items():
   print(cid, score) 
print("-----------------------------------------------------")
print("cluster 0 is the largest, cluster 1 is the second largest and cluster 2 is the smallest\n")
print("intercluster scores for every pair of cluster:")

for cid_pair,score in inter_score.items():
   print(cid_pair, score)    


intracluster scores for very cluster:
1 1140.0375066528834
0 319.1771238766518
2 3867.7247873603037
-----------------------------------------------------
cluster 0 is the largest, cluster 1 is the second largest and cluster 2 is the smallest

intercluster scores for every pair of cluster:
(1, 0) 744.3454376966346
(1, 2) 2598.1074296167694
(0, 2) 2268.718168712698
